In [14]:
from keras.models import Sequential, Model, load_model
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, BatchNormalization, \
    Input
from keras.engine.topology import merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# from image_util import crop_center
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# validation keras fix 
# ref: http://stackoverflow.com/questions/41796618/python-keras-cross-val-score-error/41841066#41841066
from keras.wrappers.scikit_learn import BaseWrapper
import copy

def custom_get_params(self, **params):
    res = copy.deepcopy(self.sk_params)
    res.update({'build_fn': self.build_fn})
    return res

BaseWrapper.get_params = custom_get_params

In [3]:
# load data
Xtrain = np.load('./data/Xtrain.npy')
Xtest = np.load('./data/Xtest.npy')
Ytrain = np.load('./data/Ytrain.npy')
Ytest = np.load('./data/Ytest.npy')

In [21]:
# netowrk settings

batch_size = 64
nb_epoch = 100
folder = './myresnet-training-checkpoint/'
load_path = folder + 'checkpoint-47-0.00.hdf5'

In [5]:
def addResidualBlock(inp, size=(3,3), subsample=(4,4), filters_conv_rate=2, name=None):
    conv_filters = int(inp.get_shape()[-1])*filters_conv_rate
    inp = BatchNormalization()(inp)
    first = Convolution2D(conv_filters, size[0], size[1], border_mode='same', subsample=subsample, activation='relu')(inp)
    
    second = BatchNormalization()(first)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    second = BatchNormalization()(second)
    second = Convolution2D(conv_filters, 3,3, border_mode='same', activation='relu')(second)
    
    return merge([first, second], mode='sum', name=name)

In [12]:
def MyResNet(weights=None):
    inp = Input(shape=(256, 256,3))
    out = Convolution2D(16, 3,3, border_mode='same', activation='relu', subsample=(4,4))(inp)
    for i in range(3):
        out = addResidualBlock(out)
        out = BatchNormalization()(out)
        out = Activation('relu')(out)
    
    out = BatchNormalization()(out)
    out = Flatten()(out)
    out = Dense(128, activation='relu', name='fcc')(out)
    out = Dense(1, name='out')(out)
    
    model = Model(input=inp, output=out)
    if weights:
        model.set_weights(weights)
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    return model


In [22]:
if load_path:
    model = load_model(load_path)
else:
    model = MyResNet()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 64, 16)    448         input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 64, 64, 16)    64          convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 16, 16, 32)    4640        batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [9]:
# batch logger 
# ref: https://github.com/fchollet/keras/issues/2850#issuecomment-222542429
class NBatchLogger(Callback):
    def __init__(self,display=100):
        '''
        display: Number of batches to wait before outputting loss
        '''
        self.seen = 0
        self.display = display

    def on_batch_end(self,batch,logs={}):
        self.seen += logs.get('size', 0)
        if self.seen % self.display == 0:
            print('\n{0}/{1} - Batch Loss: {2}'.format(self.seen,self.params['nb_sample'],
                                                self.params['metrics'][0]))

ValueError: Shapes must be equal rank, but are 4 and 0 for 'Assign' (op: 'Assign') with input shapes: [3,3,3,16], [].

In [ ]:
#train just 1000 samples, 3 epochs
# if False:
checkpoint = ModelCheckpoint(filepath=folder + 'checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
out_batch = NBatchLogger(display=1)
history = model.fit(x=Xtrain, y=Ytrain, 
          nb_epoch=nb_epoch, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint]) 

Train on 13864 samples, validate on 1541 samples
Epoch 1/100
13864/13864 [==============================] - 122s - loss: 0.1170 - val_loss: 0.0470

In [16]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [23]:
evl = model.evaluate(x=Xtest, y=Ytest,batch_size=batch_size)

3840/3852 [============================>.] - ETA: 0s 

In [24]:
evl

0.0042826834603561896

In [20]:
evl

0.002468179222131061

In [ ]:
# regression 
# rev: http://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=MyNet, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, Xtrain[:1000], Ytrain[:1000], cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))